# Main Script



----
###  ATTENZIONE 
### Ricordiamoci di committare nella cartella pickle_saves/models anche il modello 'RFR.save' (2.5gb) non l'abbiamo committato perche troppo grande
___

In [ ]:
%pip uninstall tensorflow-tensorboard -y
%pip uninstall tensorflow -y
%pip uninstall tensorflow-intel -y
%pip uninstall tensorboard -y
%pip uninstall seaborn -y
%pip uninstall keras -y
%pip uninstall pytorch_tabular -y
%pip uninstall pytorch-tabnet -y
%pip uninstall tab-transformer-pytorch -y
%pip uninstall torch -y
%pip uninstall torchvision -y
%pip uninstall torchaudio -y
%pip uninstall Pillow -y

#### Needed Imports

In [ ]:
%pip uninstall tensorflow-tensorboard
%pip uninstall tensorflow tensorflow-intel tensorboard -y

%pip install seaborn
%pip install  keras
%pip install  tensorflow
%pip install tensorflow-intel
%pip install tensorboard
#%pip install --upgrade tensorboard
%pip install pytorch_tabular pytorch-tabnet tab-transformer-pytorch
#%pip install torch torchvision torchaudio #Only CPU
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 #CUDA 11.8
%pip install Pillow

In [11]:
#Import delle Librerie
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.stats import iqr
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error,mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
import scipy.stats as stats
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
# import tensorflow as tf
# from tensorflow import keras
# from keras.layers import Dense
# from keras.models import Sequential
# from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from sklearn.model_selection import train_test_split
import pickle
from pickle import dump
import random
import os
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, Subset
from torch.utils.tensorboard.writer import SummaryWriter
from sklearn.model_selection import learning_curve
from sklearn.metrics import accuracy_score,make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from pytorch_tabular import TabularModel
from pytorch_tabular.models import TabNetModelConfig  # Importa TabNet
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
)
from pytorch_tabular.tabular_model_tuner import TabularModelTuner

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### Loading data from data_loader.py

In [82]:
# CSV zip folder's path
csv_file_name = '../data.zip'

# loading data from csv
data = pd.read_csv(csv_file_name)

In [ ]:
# CSV zip folder's path
csv_file_name = '/content/drive/MyDrive/Colab Notebooks/data.zip'

# loading data from csv
data = pd.read_csv(csv_file_name)

In [11]:
len(data)

252175

---
#### Data Acquisition

#### Describing data

In [83]:
#print shape of dataset
print("Dataset shape:",data.shape) #25175 row, 91 columns. We know that the label Y ('year) is the first one column.

# saving data without first column 'year'
no_target_column_data = data[data.columns[1:]]
print(no_target_column_data.describe())

Dataset shape: (252175, 91)
                  S0             S1             S2             S3  \
count  252175.000000  252175.000000  252175.000000  252175.000000   
mean       43.379379       1.555258       8.643927       1.160078   
std         6.066547      51.551085      35.235495      16.336577   
min         3.455260    -334.953220    -301.005060    -149.962040   
25%        39.959775     -25.651750     -11.524900      -8.492585   
50%        44.250440       8.655610      10.516440      -0.641920   
75%        47.822515      36.248650      29.792790       8.766685   
max        61.970140     384.065730     318.868960     228.412110   

                  S4             S5             S6             S7  \
count  252175.000000  252175.000000  252175.000000  252175.000000   
mean       -6.526075      -9.532049      -2.366866      -1.788645   
std        22.841967      12.809154      14.530697       7.963275   
min      -181.953370     -72.717370    -111.017810     -68.404510   
25%  

In [8]:
#Descrizione database
data.info() #Non ci sono valori mancanti e tutte le colonne dello stesso tipo (float64) ad eccezione della colonna 'Year' (int64)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252175 entries, 0 to 252174
Data columns (total 91 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Year    252175 non-null  int64  
 1   S0      252175 non-null  float64
 2   S1      252175 non-null  float64
 3   S2      252175 non-null  float64
 4   S3      252175 non-null  float64
 5   S4      252175 non-null  float64
 6   S5      252175 non-null  float64
 7   S6      252175 non-null  float64
 8   S7      252175 non-null  float64
 9   S8      252175 non-null  float64
 10  S9      252175 non-null  float64
 11  S10     252175 non-null  float64
 12  S11     252175 non-null  float64
 13  S12     252175 non-null  float64
 14  S13     252175 non-null  float64
 15  S14     252175 non-null  float64
 16  S15     252175 non-null  float64
 17  S16     252175 non-null  float64
 18  S17     252175 non-null  float64
 19  S18     252175 non-null  float64
 20  S19     252175 non-null  float64
 21  S20     25

In [84]:
target_label = data[data.columns[:1]]

#### Data by target column 'Year'

Sono presenti righe con categorie della colonna target 'Year' che vanno dal 1956 al 2009.

C'è uno sbilanciamento del dataset, che presenta molte più righe (e per cui canzoni e features su cui verranno allenati i modelli) per gli anni dal (circa) 1990 in poi.

Infatti, nel dataset per il 1990 stesso ci sono 3628 righe, per il 2000 ce ne sono più di 9mila e per il 2009 (anno maggiore == più vicino nel tempo all'anno odierno) ce ne sono più di 15mila.
In generale per gli anni 2000 ci sono circa più di 10mila righe, per gli altri, allontanandosi sempre più indietro nel tempo, ce ne sono sempre meno fino a toccare un minimodi circa 200 / 300.

E' necessario bilanciare il dataset o comunque affidare dei pesi ad ogni categoria / classe in modo tale che il training sia bilanciato in base ai dati a nostra disposizione.

In [8]:
sorted_data_by_year = data.sort_values(by='Year')

year_count = sorted_data_by_year['Year'].value_counts()
print(year_count.sort_index())

Year
1956      283
1957      299
1958      291
1959      296
1960      212
1961      285
1962      302
1963      451
1964      473
1965      560
1966      689
1967      859
1968      934
1969     1105
1970     1174
1971     1065
1972     1144
1973     1298
1974     1092
1975     1241
1976     1089
1977     1251
1978     1463
1979     1554
1980     1551
1981     1581
1982     1798
1983     1693
1984     1684
1985     1789
1986     2110
1987     2561
1988     2805
1989     3335
1990     3628
1991     4323
1992     4771
1993     5263
1994     6061
1995     6629
1996     7065
1997     7591
1998     7907
1999     9119
2000     9642
2001    10795
2002    11726
2003    13691
2004    14803
2005    17476
2006    18767
2007    19702
2008    17380
2009    15519
Name: count, dtype: int64


In [ ]:
infos=sorted_data_by_year.groupby('Year')
print(infos.describe())

---
#### Data visualization

##### Density plot

In [ ]:
#Funzione Density Plot/Histogramma per tutte le colonne del dataset

def plot_density(dataset, columns_per_row=4, figsize=(15, 10)):
   
    num_columns =data.shape[1]
    num_rows = (num_columns - 1) // columns_per_row + 1
    
    fig, axes = plt.subplots(num_rows, columns_per_row, figsize=figsize)
    axes = axes.flatten()
    
    for i, column in enumerate(dataset.columns):
        ax = axes[i]
        ax.set_title(column)
        ax.hist(dataset[column], density=True, bins=30, alpha=0.5)
        dataset[column].plot(kind='kde', ax=ax, color='blue')
        
    for ax in axes[num_columns:]:
        ax.axis('off')  # Nasconde gli assi per le eventuali celle vuote
        
    plt.tight_layout()
    plt.show()

# Utilizzo della funzione 
plot_density(data.iloc[:,1:], columns_per_row=4, figsize=(55, 50))


Dal density plot notiamo come non sia presente una distribuzione gaussiana dei dati.

##### Box plot

In [ ]:
# Box plot
plt.figure(figsize=(20, 8))
sns.boxplot(data=no_target_column_data, orient='v')  # orient='v' indica un boxplot verticale

# Aggiunta di etichette all'asse x
plt.xticks(rotation=90)  # Puoi regolare l'angolo di inclinazione delle etichette per una migliore leggibilità

# Titoli e etichette degli assi
plt.xlabel('Features')
plt.ylabel('Values')

# Visualizzazione del boxplot
plt.show()

In [ ]:
# Box plot S0 to S12
plt.figure(figsize=(10, 12))
sns.boxplot(data=no_target_column_data.iloc[:,0:13], orient='v')  # orient='v' indica un boxplot verticale

# Aggiunta di etichette all'asse x
plt.xticks(rotation=90)  # Puoi regolare l'angolo di inclinazione delle etichette per una migliore leggibilità

# Titoli e etichette degli assi
plt.xlabel('Features')
plt.ylabel('Values')

# Visualizzazione del boxplot
plt.show()

In [ ]:
# Box plot S13 to S23
plt.figure(figsize=(10, 15))
sns.boxplot(data=no_target_column_data.iloc[:,13:24], orient='v')  # orient='v' indica un boxplot verticale

# Aggiunta di etichette all'asse x
plt.xticks(rotation=90)  # Puoi regolare l'angolo di inclinazione delle etichette per una migliore leggibilità

# Titoli e etichette degli assi
plt.xlabel('Features')
plt.ylabel('Values')

# Visualizzazione del boxplot
plt.show()

In [ ]:
# Box plot S24 to S34
plt.figure(figsize=(10, 17))
sns.boxplot(data=no_target_column_data.iloc[:,24:35], orient='v')  # orient='v' indica un boxplot verticale

# Aggiunta di etichette all'asse x
plt.xticks(rotation=90)  # Puoi regolare l'angolo di inclinazione delle etichette per una migliore leggibilità

# Titoli e etichette degli assi
plt.xlabel('Features')
plt.ylabel('Values')

# Visualizzazione del boxplot
plt.show()

In [ ]:
# Box plot S35 to S45
plt.figure(figsize=(10, 17))
sns.boxplot(data=no_target_column_data.iloc[:,35:46], orient='v')  # orient='v' indica un boxplot verticale

# Aggiunta di etichette all'asse x
plt.xticks(rotation=90)  # Puoi regolare l'angolo di inclinazione delle etichette per una migliore leggibilità

# Titoli e etichette degli assi
plt.xlabel('Features')
plt.ylabel('Values')

# Visualizzazione del boxplot
plt.show()

In [ ]:
# Box plot S46 to S54
plt.figure(figsize=(10, 12))
sns.boxplot(data=no_target_column_data.iloc[:,46:55], orient='v')  # orient='v' indica un boxplot verticale

# Aggiunta di etichette all'asse x
plt.xticks(rotation=90)  # Puoi regolare l'angolo di inclinazione delle etichette per una migliore leggibilità

# Titoli e etichette degli assi
plt.xlabel('Features')
plt.ylabel('Values')

# Visualizzazione del boxplot
plt.show()

In [ ]:
# Box plot S55 to S64
plt.figure(figsize=(10, 12))
sns.boxplot(data=no_target_column_data.iloc[:,55:65], orient='v')  # orient='v' indica un boxplot verticale

# Aggiunta di etichette all'asse x
plt.xticks(rotation=90)  # Puoi regolare l'angolo di inclinazione delle etichette per una migliore leggibilità

# Titoli e etichette degli assi
plt.xlabel('Features')
plt.ylabel('Values')

# Visualizzazione del boxplot
plt.show()

In [ ]:
# Box plot S65 to S75
plt.figure(figsize=(10, 12))
sns.boxplot(data=no_target_column_data.iloc[:,65:76], orient='v')  # orient='v' indica un boxplot verticale

# Aggiunta di etichette all'asse x
plt.xticks(rotation=90)  # Puoi regolare l'angolo di inclinazione delle etichette per una migliore leggibilità

# Titoli e etichette degli assi
plt.xlabel('Features')
plt.ylabel('Values')

# Visualizzazione del boxplot
plt.show()

In [ ]:
# Box plot S76 to S82
plt.figure(figsize=(10, 12))
sns.boxplot(data=no_target_column_data.iloc[:,76:83], orient='v')  # orient='v' indica un boxplot verticale

# Aggiunta di etichette all'asse x
plt.xticks(rotation=90)  # Puoi regolare l'angolo di inclinazione delle etichette per una migliore leggibilità

# Titoli e etichette degli assi
plt.xlabel('Features')
plt.ylabel('Values')

# Visualizzazione del boxplot
plt.show()

In [ ]:
# Box plot S83 to S89
plt.figure(figsize=(10, 12))
sns.boxplot(data=no_target_column_data.iloc[:,83:90], orient='v')  # orient='v' indica un boxplot verticale

# Aggiunta di etichette all'asse x
plt.xticks(rotation=90)  # Puoi regolare l'angolo di inclinazione delle etichette per una migliore leggibilità

# Titoli e etichette degli assi
plt.xlabel('Features')
plt.ylabel('Values')

# Visualizzazione del boxplot
plt.show()

Nei **Boxplots** che abbiamo mostrato qua sopra, si possono notare come le features hanno una distribuzione di dati simile per quasi tutte le colonne, tranne per quelle che vanno da: *S0* a *S12*. Sono presenti, per ogni features, un gran numero di *outliers* sia sotto i valori minimi sia sopra i valori massimi del boxplot.



##### Correlation Matrix

In [ ]:
# Calcolare la matrice di correlazione
correlation_matrix = no_target_column_data.corr()

# Creare una heatmap della matrice di correlazione usando seaborn
plt.figure(figsize=(20, 20))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Matrice di Correlazione')
plt.show()

In [ ]:
# Eliminare i coefficienti di correlazione uguali a 1
correlation_matrix = correlation_matrix.mask(correlation_matrix == 1)

# Estrarre le prime 20 coppie con i coefficienti di correlazione più alti
top_corr_pairs = correlation_matrix.unstack().nlargest(40)

# Stampa i nomi delle colonne delle prime 20 coppie
print("Nomi delle colonne delle prime 20 coppie con coefficiente di correlazione più alto:")
for (var1, var2), corr in top_corr_pairs.items():
    print(f"{var1} e {var2}, Correlazione: {corr}")
    print(f"Nomi delle colonne: {var1}, {var2}")
    print()

##### Correlation matrix su tutto il dataset
Data la correlation matrix riportata sopra abbiamo notato come vi sia una forte relazione tra le colonne da S12 a S24 circa, che sono caratterizzate da una forte presenza di valori outlier (lo abbiamo visto dai boxplot).
Dunque, proviamo anche a calcolare la correlation matrix escludendo queste colonne.

In [ ]:
subset_data = pd.concat([no_target_column_data.iloc[:,:12], no_target_column_data.iloc[:,24:]], axis=1)

# Calcolare la matrice di correlazione
correlation_matrix = subset_data.corr()

# Creare una heatmap della matrice di correlazione usando seaborn
plt.figure(figsize=(20, 20))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Matrice di Correlazione')
plt.show()

In [ ]:
# Eliminare i coefficienti di correlazione uguali a 1
correlation_matrix = correlation_matrix.mask(correlation_matrix == 1)

# Estrarre le prime 20 coppie con i coefficienti di correlazione più alti
top_corr_pairs = correlation_matrix.unstack().nlargest(40)

# Stampa i nomi delle colonne delle prime 20 coppie
print("Nomi delle colonne delle prime 20 coppie con coefficiente di correlazione più alto:")
for (var1, var2), corr in top_corr_pairs.items():
    print(f"{var1} e {var2}, Correlazione: {corr}")
    print(f"Nomi delle colonne: {var1}, {var2}")
    print()

##### Correlation matrix (S12:S24 escluse)
In questa correlation matrix notiamo come i coefficienti di correlazione siano effettivamente più bassi, evidenziando il fatto che le variabili non sono correlate nello stesso modo in cui lo sono quelle dalla 12 alla 24.

Dobbiamo dunque trattare gli outlier prima di poter stabilire quali variabili siano più rilevanti rispetto alle altre.

##### Conclusioni dopo data visualization
Dal processo di visualizzazione dei dati deduciamo che è necessario trattare gli *outliers*: consideriamo quindi che non siano rilevanti per l'allenamento del nostro modello, ma si tratti di dati rumorosi e fuorvianti.

In ogni caso valutare allenamento modelli con outliers.

Valutare problema di **High Dimensionality**: tramite PCA (Princial Component Analysis) e LDA (Linear Discrimination Analysis)

Valutare **Normalizzazione**: vedendo la distribuzione non gaussiana, opterei per **MinMaxScaling** (non escludere *Standardization*)

##### Trattamento outliers by winsorization

In [ ]:
no_target_column_smoothed_data = pd.DataFrame(no_target_column_data.copy())

def winsorize_column(column, k=1.5):
    q1 = column.quantile(0.25)
    q3 = column.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - k * iqr
    upper_bound = q3 + k * iqr
    column = np.where(column < lower_bound, lower_bound, column)
    column = np.where(column > upper_bound, upper_bound, column)
    return column

for column in no_target_column_smoothed_data.columns:
  no_target_column_smoothed_data[column] = winsorize_column(no_target_column_smoothed_data[column])

#### Z-SCORE (sostituzione outliers con MEDIAN) : MSE (post MinMaxScaling) : 41.132

In [ ]:
def replace_outliers_with_median(dataset, threshold=3):
    # Copia del dataset per evitare modifiche indesiderate
    cleaned_dataset = dataset.copy()

    # Itera su ogni colonna/feature del dataset
    for column in cleaned_dataset.columns:
        # Calcola lo z-score per la feature corrente
        z_scores = np.abs(stats.zscore(cleaned_dataset[column]))

        # Trova gli outliers superando la soglia
        outliers_indices = np.where(z_scores > threshold)[0]

        # Sostituisci gli outliers con la mediana della colonna
        column_median = cleaned_dataset[column].median()
        cleaned_dataset[column].iloc[outliers_indices] = column_median

    return cleaned_dataset

In [ ]:
# Sostituisci gli outliers con la media delle colonne
data_rmoutliers = replace_outliers_with_mean(data, 2)

In [ ]:
data_rmoutliers.shape

In [ ]:
data_rmoutliers.isna().sum().sum() #Restituisce il Num totale dei valori NaN

In [ ]:
X = no_target_column_smoothed_data[no_target_column_smoothed_data.columns[1:]]
y = no_target_column_smoothed_data[no_target_column_smoothed_data.columns[:1]]

#### Preprocessing: suddivisione del dataset

In [85]:
X=no_target_column_data.copy() #90 COLONNE
y=target_label.copy()

#### Splitting Dataset: Train & Validation

In [86]:
# X sono le feature (variabili indipendenti), y è la colonna target
# 80% training set - 20% validation set
seed=89
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=seed,stratify=y)

In [21]:
import numpy as np

def find_nan_positions(data):
    nan_positions = np.argwhere(pd.isna(data).to_numpy())
    return nan_positions

nan_positions = find_nan_positions(X_train)

if nan_positions.size > 0:
    print("Valori NaN trovati alle seguenti posizioni:")
    for position in nan_positions:
        print(f"Riga: {position[0] + 1}, Colonna: {position[1] + 1}")
else:
    print("Il DataFrame non contiene valori NaN.")


Il DataFrame non contiene valori NaN.


---
#### Data Preprocessing (LR)


In [87]:
X = X_train.copy()
y = y_val.copy()

##### Boxplot after outliers' smoothing

In [ ]:
# Box plot
plt.figure(figsize=(20, 8))
sns.boxplot(data=X, orient='v')  # orient='v' indica un boxplot verticale

# Aggiunta di etichette all'asse x
plt.xticks(rotation=90)  # Puoi regolare l'angolo di inclinazione delle etichette per una migliore leggibilità

# Titoli e etichette degli assi
plt.xlabel('Features')
plt.ylabel('Values')

# Visualizzazione del boxplot
plt.show()

#### Undersampling e Oversampling (non migliora MSE) 

In [ ]:
from collections import Counter
from imblearn.under_sampling import ClusterCentroids
from imblearn.over_sampling import SMOTE

def downSampling(x_train,y_train):
    # Controlla la distribuzione delle classi prima del downsampling
    print("Distribuzione delle classi prima del downsampling:")
    print(Counter(y_train['Year']))

    cc = ClusterCentroids(random_state=seed,)

    # Effettua il downsampling
    X_resampled, y_resampled = cc.fit_resample(X, y_train['Year'])

    return X_resampled,y_resampled

def upSampling(x_train,y_train):
    X_resampled, y_resampled = SMOTE().fit_resample(x_train, y_train['Year'])
    return X_resampled,y_resampled


x_resempled,y_resampled= upSampling(X_train,y_train)


##### Min max scaling

In [88]:
X_Validation = X_val.copy()

# Min-Max Scaling
# Creazione dell'oggetto MinMaxScaler e adattamento solo al training set
#file = open("scaler.save","wb") #salvataggio dello scaler sul disco nel file "scaler.save"
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

#apply transform on validation set
X_val_scaled = scaler.transform(X_Validation)
#pickle.dump(scaler, file)
#file.close()

# save the model
with open("../pickle_saves/preprocess/minMaxScaler.save", "wb") as file:
    pickle.dump(scaler, file)

#name columns after min-max scaling
# num_colonne = X_scaled.shape[1]  # Ottieni il numero di colonne
# nome_colonne = ['S' + str(i) for i in range(num_colonne)]
# X_scaled_df = pd.DataFrame(X_scaled, columns=nome_colonne)

# print("Dati di training originali:")
# print(X)
print("\nDati di training normalizzati MinMaxScaling:")
print(X_scaled)
# print("\nDati di validation originali:")
# print(X_Validation)
# print("\nDati di validation normalizzati Min-Max:")
# print(X_val_scaled)

# save the model
#with open("../pickle_saves/models/minMaxScaler.save", "wb") as file:
#    pickle.dump(scaler, file)


Dati di training normalizzati MinMaxScaling:
[[0.60871594 0.53356923 0.4494677  ... 0.53845299 0.51108995 0.35777756]
 [0.56156007 0.50000065 0.67651561 ... 0.47862945 0.52086774 0.37412128]
 [0.7086659  0.56483163 0.51686467 ... 0.49568095 0.50430072 0.33343394]
 ...
 [0.62054814 0.41932825 0.42143407 ... 0.50094978 0.51898317 0.34341102]
 [0.76981599 0.53818665 0.51157919 ... 0.49243993 0.50699643 0.31996933]
 [0.54791055 0.34766547 0.56625569 ... 0.54115006 0.50531332 0.40491751]]


#### Standardization

In [ ]:
# standardization of dependent variables
scaler=StandardScaler()
#standardization x_train
X_std = scaler.fit_transform(X) 
#apply transform on validation set
X_val_std = scaler.transform(X_Validation)

##### Boxplot after min max scaling

In [ ]:
# Box plot
plt.figure(figsize=(20, 8))
sns.boxplot(data=X_scaled, orient='v')  # orient='v' indica un boxplot verticale

# Aggiunta di etichette all'asse x
plt.xticks(rotation=90)  # Puoi regolare l'angolo di inclinazione delle etichette per una migliore leggibilità

# Titoli e etichette degli assi
plt.xlabel('Features')
plt.ylabel('Values')

# Visualizzazione del boxplot
plt.show()

##### Density plot after min max scaling

In [ ]:
# Density plot
fig, axs = plt.subplots(15, 3, figsize=(15, 60))
counter = 0

for i in range(15):
  for j in range(3):

    # Plot KDE
    sns.kdeplot(X_scaled[:,counter], ax=axs[i, j], color='g', bw_adjust=2, label='Smooth factor=2')

    # Plot CDF
    sns.kdeplot(X_scaled[:,counter], ax=axs[i, j], color='r', cumulative=True, label='CDF')

    axs[i, j].set_xlabel("S" + str(counter))
    counter = counter+1

plt.show()

##### PCA

##### Studio n_components

In [ ]:
num_columns = X_scaled.shape[1]
nums = np.arange(num_columns)

In [ ]:
var_ratio = []
for num in nums:
  pca = PCA(n_components=num)
  pca.fit(X_scaled)
  var_ratio.append(np.sum(pca.explained_variance_ratio_))

In [ ]:
#Dalla Figura si nota come la linea di cutoff del 95% interseca il valore della X nella colonna 51/52
plt.figure(figsize=(30,4),dpi=150)
plt.grid()
plt.plot(nums,var_ratio,marker='o')
plt.xlabel('n_components')
plt.xticks(np.arange(0,90,1))
plt.ylabel('Explained variance ratio')
plt.title('n_components vs. Explained Variance Ratio')
#linea di cutoff del 95%
plt.axhline(y=0.95, color='r', linestyle='-')
plt.text(0.5, 0.85, '95% cut-off threshold', color = 'red', fontsize=16)

##### PCA Analysis

In [39]:
# Inizializzare l'oggetto PCA
pca = PCA(n_components=52) #52
X_decomposed = pca.fit_transform(X_scaled)
#pca.get_feature_names_out(X_scaled_df.columns)

# Ottieni i vettori delle componenti principali
components = pca.components_


#validation data after PCA
X_val_decomposed = pca.transform(X_val_scaled)

with open("../pickle_saves/preprocess/pca.save", "wb") as file:
    pickle.dump(pca, file)


In [ ]:
#Scree Plot -> mostra la percentuale di variazione spiegata da ogni componente principale
PC_values = np.arange(pca.n_components_) + 1
plt.figure(figsize=(10,10))
plt.grid()
plt.plot(PC_values, pca.explained_variance_ratio_, 'o-', linewidth=2, color='blue')
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.xticks(range(0,90,2))
plt.ylabel('Variance Explained')
plt.show()
print("Explainer variance of each component:",pca.explained_variance_ratio_)
print("Variance total:", sum(pca.explained_variance_ratio_))

In [ ]:
num_columns=X_decomposed.shape[1]

##### Boxplot after PCA

In [ ]:
# Box plot
plt.figure(figsize=(20, 8))
sns.boxplot(data=X_decomposed, orient='v')  # orient='v' indica un boxplot verticale

# Aggiunta di etichette all'asse x
plt.xticks(rotation=90)  # Puoi regolare l'angolo di inclinazione delle etichette per una migliore leggibilità

# Titoli e etichette degli assi
plt.xlabel('Features')
plt.ylabel('Values')

# Visualizzazione del boxplot
plt.show()

#### Regressione Lineare

In [62]:
##MinMaxScaling(), MSE: 84.500
##MinMaxScaling(),PCA, MSE:86.114

regr = LinearRegression()

# Train the model using the training sets
regr.fit(X_scaled, y_train)


# save the model
with open("../pickle_saves/models/LR.save", "wb") as file:
    pickle.dump(regr, file)

# Make predictions using the testing set
y_predictions = regr.predict(X_val_scaled)
#MSE
mse = mean_squared_error(y_val, y_predictions)
#MAE
mae = mean_absolute_error(y_val, y_predictions)
#MAPE
mape = mean_absolute_percentage_error(y_val, y_predictions)
#R2
r2 = r2_score(y_val, y_predictions)

# # save the model
# with open("../pickle_saves/models/regr.save", "wb") as file:
#     pickle.dump(regr, file)

print("Mean squared error: %.3f" % mse)
print("MAE: %.3f" % mae)
print("MAPE: %.3f" % mape)
print("R2: %.3f" % r2)

Mean squared error: 84.343
MAE: 6.603
MAPE: 0.003
R2: 0.235


#### Testing LR

In [71]:
model=pickle.load(open("../pickle_saves/models/LR.save", 'rb'))
scaler=pickle.load(open("../pickle_saves/preprocess/minMaxScaler.save", 'rb'))
X_scaled=scaler.transform(no_target_column_data)
y_predictions=model.predict(X_scaled)

#MSE
mse = mean_squared_error(target_label, y_predictions)
#MAE
mae = mean_absolute_error(target_label, y_predictions)
#MAPE
mape = mean_absolute_percentage_error(target_label, y_predictions)
#R2
r2 = r2_score(target_label, y_predictions)

# # save the model
# with open("../pickle_saves/models/regr.save", "wb") as file:
#     pickle.dump(regr, file)

print("Mean squared error: %.3f" % mse)
print("MAE: %.3f" % mae)
print("MAPE: %.3f" % mape)
print("R2: %.3f" % r2)

Mean squared error: 84.429
MAE: 6.623
MAPE: 0.003
R2: 0.234


#### Random Forest Regressor con GridSearch

In [ ]:
# Definire la grid e la metrica di valutazione
grid1 = {
    'max_depth': [200,300],
    # 'max_features': [3],
    # 'min_samples_leaf': [2],
    # 'min_samples_split': [3],
    'n_estimators': [300,500,1000]
}

# Creare la scorning function
scoring = make_scorer(mean_squared_error, greater_is_better=False)

# Grid Search function
CV_rfr = GridSearchCV(estimator=RandomForestRegressor(), param_grid=grid1, cv=5, 
                      scoring="neg_mean_squared_error",n_jobs=-1,verbose=2)

# Modificare la forma del target y
y_train_reshaped = np.ravel(y_train)

CV_rfr.fit(X_scaled, y_train_reshaped)

with open("../pickle_saves/models/RFR.save", "wb") as file:
    pickle.dump(CV_rfr.best_params_, file)

predictions = CV_rfr.predict(X_val_scaled)
mse = mean_squared_error(y_val, predictions)



# Print results
print("----------BEST PARAMS----------")
print(CV_rfr.best_params_)
print("--------------------------------")
print("Mean squared error: %.3f" % mse)

# save the model --- Runnare il modell RFR 
# with open("../pickle_saves/models/RFR.save", "wb") as file:
#     pickle.dump(CV_rfr.best_params_, file)

#-------------MINMAXSCALING (CON PCA=80)------------------
#----------BEST PARAMS----------
#{'max_depth': 4, 'n_estimators': 300, 'random_state': 89}
#Mean squared error: 93.907
#-------------MINMAXSCALING (CON PCA=52)------------------
# ----------BEST PARAMS----------
# {'max_depth': 4, 'n_estimators': 1000, 'random_state': 89}
# --------------------------------

#-----------Solo MinMaxScaling------------------
# Mean squared error: 93.910
# ----------BEST PARAMS----------
# {'max_depth': 90, 'max_features': 3, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 300}
# --------------------------------
# Mean squared error: 84.530
# ----------BEST PARAMS----------
# {'max_depth': 200, 'max_features': 3, 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 300}
# --------------------------------
# Mean squared error: 84.378

##### Random Forest Best MODEL

In [92]:
rfr=RandomForestRegressor(max_depth=200,max_features=3, min_samples_leaf= 2, min_samples_split= 3, n_estimators= 300)
# Modificare la forma del target y
y_train_reshaped = np.ravel(y_train)

rfr.fit(X_scaled, y_train_reshaped)

#Salvataggio Modello RFR
with open("../pickle_saves/models/RFR.save", "wb") as file:
    pickle.dump(rfr, file)

y_predictions = rfr.predict(X_val_scaled)

#MSE
mse = mean_squared_error(y_val, y_predictions)
#MAE
mae = mean_absolute_error(y_val, y_predictions)
#MAPE
mape = mean_absolute_percentage_error(y_val, y_predictions)
#R2
r2 = r2_score(y_val, y_predictions)

print("Mean squared error: %.3f" % mse)
print("MAE: %.3f" % mae)
print("MAPE: %.3f" % mape)
print("R2: %.3f" % r2)

Mean squared error: 84.552
MAE: 6.817
MAPE: 0.003
R2: 0.233


#### KNN con GridSearch (Provare con Dati Normalizzati, Rimuovendo Outliers e PCA)

In [ ]:
# Definisci i parametri da testare
# param_grid = {'n_neighbors': [3, 5, 7, 9, 11]} vecchia iterazione
param_grid = {'n_neighbors': list(range(25, 60)), "weights": ["uniform","distance"],"n_jobs":[-1]} #-1 means using all processors.

# Crea un'istanza del modello KNN per la regressione
knn_regressor = KNeighborsRegressor()
y_train_new = np.ravel(y_train)

# Crea un'istanza di GridSearchCV per trovare il miglior parametro
grid_search = GridSearchCV(knn_regressor, param_grid, cv=5, scoring='neg_mean_squared_error')

# Esegui la ricerca dei parametri utilizzando il set di addestramento
grid_search.fit(X_scaled, y_train_new)

# Ottieni il miglior parametro trovato
best_k = grid_search.best_params_['n_neighbors']
best_weight = grid_search.best_params_['weights']

# Utilizza il modello addestrato con il miglior parametro per fare previsioni su nuovi dati
best_knn_regressor = grid_search.best_estimator_
y_predictions = best_knn_regressor.predict(X_val_scaled)


# scores = []
# for i in range(1,100):
#     knn = KNeighborsRegressor(n_neighbors=i)
#     knn.fit(X_scaled,y_train_new)
#     y_pred = knn.predict(X_val_scaled)
#     scores.append(accuracy_score(y_val, y_pred))

# plt.plot(range(1,16),scores)

# Calcola le metriche di valutazione
mse = mean_squared_error(y_val, y_predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_predictions)
mae = mean_absolute_error(y_val, y_predictions)
#MAPE
mape = mean_absolute_percentage_error(y_val, y_predictions)

print("Miglior parametro K:", best_k, "weights:",best_weight)
print("MSE:", mse)
print("RMSE:", rmse)
print("R2 Score:", r2)
print("MAE:", mae)
print("MAPE: %.3f" % mape)
# BEST PARAMS
# Miglior parametro K: 28 weights: distance
# MSE: 76.07494414444997

# save the model --- runnare modello migliore, non la gridsearch
with open("../pickle_saves/models/KNN.save", "wb") as file:
    pickle.dump(knn_regressor, file)

#### KNN Best Model

In [94]:
# Crea un'istanza del modello KNN per la regressione
knn_regressor = KNeighborsRegressor(n_neighbors=28,weights='distance')
y_train_new = np.ravel(y_train)

#train del modello knnr
knn_regressor.fit(X_scaled,y_train_new)

#predizioni 
y_predictions = knn_regressor.predict(X_val_scaled)

# Calcola le metriche di valutazione
mse = mean_squared_error(y_val, y_predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_predictions)
mae = mean_absolute_error(y_val, y_predictions)
#MAPE
mape = mean_absolute_percentage_error(y_val, y_predictions)

#print("Miglior parametro K:", best_k, "weights:",best_weight)
print("MSE:", mse)
print("RMSE:", rmse)
print("R2 Score:", r2)
print("MAE:", mae)
print("MAPE: %.3f" % mape)

MSE: 76.07494414444997
RMSE: 8.722095169421735
R2 Score: 0.30976056378715877
MAE: 6.315206874447766
MAPE: 0.003


#### SVM Regressor

In [ ]:
# Definizione della griglia di iperparametri e kernel
#Eseguire cosi provare
param_grid = {
    'C': [0.1,0.8,1,1.5,10],
    'epsilon': [0.001,0.01,0.5,0.8,1],
    'kernel': ['poly','rbf','sigmoid']
}
#Settaggio: 'C': 0.1, 'epsilon': 0.01, 'kernel': 'poly'}
# MSE: 86.91330648460543
# RMSE: 9.322730634562248
# R2 Score: 0.21142246843562673
# MAE: 6.22451918257399
# MAPE: 0.003128732327132059
#----------------
#{'C': 0.5, 'epsilon': 0.1, 'kernel': 'poly'} 
#MSE: 84.39535038972714


#Best parameters------ {'C': 1, 'epsilon': 0.5, 'kernel': 'poly'
# GRID SEARCH
# 'C': [1],
#'epsilon': [0.1,0.5],
#'kernel': ['poly']

#MSE: 83.466370627596
#RMSE: 9.135993138547994
#R2 Score: 0.24269703707791712
#MAE: 6.073974985117682
#MAPE: 0.0030530406859256344

#Best parameters: {'C': 1, 'epsilon': 0.8, 'kernel': 'poly'}-- Meglio
#-----------GRID SEARCH-----------
# 'C': [0.1,1],
#     'epsilon': [0.5,0.8],
#     'kernel': ['poly']
#--------------------------------
#MSE: 83.40573931226207
#RMSE: 9.132674269471242
#R2 Score: 0.2432471541418677
#MAE: 6.07423337170572
#MAPE: 0.0030531516336399058

y_train_new = np.ravel(y_train)

# Creazione del modello SVR
svr_model = SVR()

# Ricerca Grid per il tuning degli iperparametri e dei kernel
grid_search = GridSearchCV(estimator=svr_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error',n_jobs=-1)
grid_search.fit(X_scaled, y_train_new)

# Valutazione del modello migliore
best_model = grid_search.best_estimator_
y_predictions = best_model.predict(X_val_scaled)

# Calcola MSE
mse = mean_squared_error(y_val, y_predictions)
# Calcola RMSE
rmse = np.sqrt(mse)
# Calcola R2 Score
r2 = r2_score(y_val, y_predictions)
# Calcola MAE
mae = mean_absolute_error(y_val, y_predictions)
# Calcola MAPE
mape = mean_absolute_percentage_error(y_val,y_predictions)

print("Best parameters:", grid_search.best_params_)
print("MSE:", mse)
print("RMSE:", rmse)
print("R2 Score:", r2)
print("MAE:", mae)
print("MAPE:", mape)

# save the model --- RUNNA IL MODELLO
with open("../pickle_saves/models/SVR.save", "wb") as file:
    pickle.dump(svr_model, file)

In [97]:

# Creazione del modello SVR
svr_model = SVR(C=1, epsilon=0.8, kernel='poly')
svr_model.fit(X_scaled,y_train_new)

# save the model SVR
with open("../pickle_saves/models/SVR.save", "wb") as file:
    pickle.dump(svr_model, file)

#reshape y_train
y_train_new = np.ravel(y_train)

# Calcola MSE
mse = mean_squared_error(y_val, y_predictions)
# Calcola RMSE
rmse = np.sqrt(mse)
# Calcola R2 Score
r2 = r2_score(y_val, y_predictions)
# Calcola MAE
mae = mean_absolute_error(y_val, y_predictions)
# Calcola MAPE
mape = mean_absolute_percentage_error(y_val,y_predictions)

print("MSE:", mse)
print("RMSE:", rmse)
print("R2 Score:", r2)
print("MAE:", mae)
print("MAPE:", mape)

MSE: 76.07494414444997
RMSE: 8.722095169421735
R2 Score: 0.30976056378715877
MAE: 6.315206874447766
MAPE: 0.0031687451178864286


#### FF (Feed-Forward) reti neurali (Gabri)

### Tab Net Neural Network - Define structure (MIGLIORE)

In [98]:
X= data[data.columns[1:]]
Y=data[data.columns[0]]

# `num_col_names` lista con i nomi delle 89 colonne numeriche
num_col_names =X.columns.tolist()

X_train, X_test, y_train, y_test  = train_test_split(X, Y, test_size=0.1, random_state=seed,shuffle=True)
X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.25, random_state=seed,shuffle=True)  # 0.9 x 0.25 = 0.22 di validation

#Scaling dei dati MinMaxScaler()
scaler=MinMaxScaler()
X_scaled=scaler.fit_transform(X_train)
X_val_scaled=scaler.transform(X_val)
X_test_scaled=scaler.transform(X_test)

# MinMaxScaling e creazione DataFrame per train,validation,test
train = pd.DataFrame(X_scaled, columns=X.columns)
train['Year'] = y_train.values

val = pd.DataFrame(X_val_scaled, columns=X.columns)
val['Year'] = y_val.values

test = pd.DataFrame(X_test_scaled, columns=X.columns)
test['Year'] = y_test.values

# Configurazione dei dati
data_config = DataConfig(
    target=["Year"],  # target should always be a list
    continuous_cols=num_col_names,
    categorical_cols=[],
    normalize_continuous_features=False
)

#Con batch_size: 128 MSE=128
# Configurazione del trainer
trainer_config = TrainerConfig(
    #auto_lr_find=True,  # Runs the LRFinder to automatically derive a learning rate
    batch_size=32, #1024
    max_epochs=100,
    early_stopping_patience=5,  # Numero di epoche di attesa per il miglioramento
    precision=32,
    seed=seed
)

# Configurazione dell'ottimizzatore
optimizer_config = OptimizerConfig(
    optimizer="AdamW"
)

# Configurazione del modello TabNet
model_config = TabNetModelConfig(
    task="regression",
    n_d=16,
    n_a=16,
    n_steps=10,
    gamma=1.5,
    n_independent=2,
    n_shared=2,
    #virtual_batch_size=128
    seed=seed
)

# Crea il modello tabular
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

2024-06-24 17:59:18,042 - {pytorch_tabular.tabular_model:140} - INFO - Experiment Tracking is turned off

### TABNET INCASINATA

In [41]:
#volendo anche:
#model_config__n_d: range(4,65).tolist()
#model_config__n_a: range(4,65).tolist()
#model_config__n_steps: range(3,11).tolist()
#model_config__gamma: [1.0,1.2,1.5,1.8,2.0]

#BEST MSE per ora: {'optimizer_config__optimizer': 'Adam', 'trainer_config__batch_size': 24,                                          
#'trainer_config__early_stopping_patience': 3, 'trainer_config__max_epochs': 10, 'trainer_config__precision': 64,   
#'mean_squared_error': 134.18800048828126, 'trial_id': 2} | Score: 134.18800048828126  

search_space={
"trainer_config__batch_size": [32,64,128,256],
"trainer_config__max_epochs": [20],
"trainer_config__early_stopping_patience": [10],
"trainer_config__precision": [32,16],
"optimizer_config__optimizer": ["Adam", "RAdam", "AdamW"],
"optimizer_config__learning_rate": [1e-3],
}

X= data[data.columns[1:]]#.astype("float32") #Tutte le features numeriche
Y=data[data.columns[0]] # target column

# `num_col_names` lista con i nomi delle 89 colonne numeriche
num_col_names =X.columns.tolist()

X_train, X_test, y_train, y_test  = train_test_split(X, Y, test_size=0.1, random_state=seed,shuffle=True)
X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.25, random_state=seed,shuffle=True)  # 0.9 x 0.25 = 0.22 di validation

#Scaling dei dati MinMaxScaler()
scaler=MinMaxScaler()
X_scaled=scaler.fit_transform(X_train)
X_val_scaled=scaler.transform(X_val)
X_test_scaled=scaler.transform(X_test)

# MinMaxScaling e creazione DataFrame per train,validation,test
train = pd.DataFrame(X_scaled, columns=X.columns)
train['Year'] = y_train.values

val = pd.DataFrame(X_val_scaled, columns=X.columns)
val['Year'] = y_val.values

test = pd.DataFrame(X_test_scaled, columns=X.columns)
test['Year'] = y_test.values


# Configurazione dei dati
data_config = DataConfig(
    target=["Year"],  # Target deve essere una lista
    continuous_cols=num_col_names,
    categorical_cols=[],  # Nessuna colonna categoriale
    normalize_continuous_features=False, #Di default scala le feature con StandarScaler, se settato a True
    #num_workers=11 #Per Windows sempre  settato a 0
)

# Configurazione del trainer
trainer_config = TrainerConfig(
    auto_lr_find=True, # attiva
    batch_size=128, 
    max_epochs=50,
    early_stopping_patience=3,  # Numero di epoche di attesa per il miglioramento
    precision=32, 
    check_val_every_n_epoch=1,  # Frequenza di validazione
    seed=seed
)

# Configurazione dell'ottimizzatore
optimizer_config = OptimizerConfig()

#https://pytorch-tabular.readthedocs.io/en/stable/models/#tabnet
# Configurazione del modello TabNet per la regressione
model_config = TabNetModelConfig(
    task="regression",
    n_d=24,  # Dimensioni delle feature embeddings per il decoder  (usually between 4 and 64)
    n_a=24,  # Dimensioni delle feature embeddings per l'attenzione  (usually between 4 and 64)
    n_steps=5,  # Numero di decision steps  (usually between 3 and 10)
    gamma=1.5,  # Coefficiente per il modulo attuale delle feature
    n_independent=2,  # Numero di layer completamente connessi indipendenti
    n_shared=2,  # Numero di layer completamente connessi condivisi
    seed=seed,
    #Target range for Regression: https://pytorch-tabular.readthedocs.io/en/stable/models/#tabnet
    #moltiplica per 0.8 e 1.2 per espandere i valori di minimo e max degli intervalli, così da non rendere la predizione troppo stringente su quei valori (Togliere se non ha senso)
    target_range=[(round(train['Year'].min() * 0.8), round(train['Year'].max() * 1.2))] 
)


tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)


2024-06-23 23:15:04,148 - {pytorch_tabular.tabular_model:140} - INFO - Experiment Tracking is turned off

### Train Model NO GridSearch

In [99]:
# `train` e `val`  DataFrames di training e validation
tabular_model.fit(train=train, validation=val, seed=seed)

tabular_model.save_model("../pickle_saves/models/tabmodel.save")

# Valutare il modello
result = tabular_model.evaluate(val)
print(result)

# Fare predizioni
pred_df = tabular_model.predict(val)
print(pred_df)


Seed set to 89


2024-06-24 17:59:30,734 - {pytorch_tabular.tabular_model:524} - INFO - Preparing the DataLoaders

2024-06-24 17:59:30,781 - {pytorch_tabular.tabular_datamodule:499} - INFO - Setting up the datamodule for          
regression task

2024-06-24 17:59:30,918 - {pytorch_tabular.tabular_model:574} - INFO - Preparing the Model: TabNetModel

2024-06-24 17:59:31,091 - {pytorch_tabular.tabular_model:340} - INFO - Preparing the Trainer

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

c:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │       71.4716796875       │
│  test_mean_squared_error  │       71.4716796875       │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 71.4716796875, 'test_mean_squared_error': 71.4716796875}]
       Year_prediction
0          1985.046143
1          2002.721924
2          2002.949097
3          2001.316406
4          2006.157471
...                ...
56735      1998.776489
56736      2004.991455
56737      2002.020264
56738      1999.375244
56739      2000.655273

[56740 rows x 1 columns]


In [38]:
y_true = val['Year']
y_pred = pred_df['Year_prediction']

mse = mean_squared_error(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)
mape = mean_absolute_percentage_error(y_true, y_pred)
r2score = r2_score(y_true, y_pred)
print("MSE: ", mse, " MAE:", mae, " MAPE:", mape, " R2_SCORE:", r2score)

MSE:  71.47168434478702  MAE: 5.877912504285584  MAPE: 0.0029513839182513384  R2_SCORE: 0.34547632932662964


In [103]:
# Salvataggio del modello
with open('../pickle_saves/models/tabular_model.save', 'wb') as f:
    pickle.dump(tabular_model, f)

### GridSearch

In [ ]:
import warnings

tuner = TabularModelTuner(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config
)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    result = tuner.tune(
        train=train,
        validation=val,
        search_space=search_space,
        strategy="grid_search",
        cv=5, # Uncomment this to do a 5 fold cross validation 
        metric="mean_squared_error",
        mode="min",
        progress_bar=True,
        random_state=seed,
        verbose=True # Make True if you want to log metrics and params each iteration
    )

In [ ]:
#I risultati sono tupla con nome: trials_df, best_params, best_score and best_model
result.trials_df.head()
print("Best Score: ", result.best_score)
print(result.best_params)

---
#### Data Modeling (tuning hyperparams)


---
#### Performance Evaluation
